In [33]:
import matplotlib.pyplot as plt
import json

%matplotlib inline

In [34]:
# Load main data

import os
from dataclasses import dataclass
import numpy as np


@dataclass
class Data:
    time: float
    psnr: float
    legal: bool
    stage: str
    ans: np.ndarray


levels = range(7)
qscales = [0.3, 0.4, 0.5, 0.6, 0.7]
timepoint = range(0, 600, 10)

base_dir = (
    "/home/xyhang/projects/VCIP2023-grand-challenge/experiments/SAv1/visualize/"
)

# Load bpp info

result_file = "results_visualize.json"

with open(result_file, "r") as f:
    result_data = json.load(f)

In [35]:
import matplotlib

In [36]:
filenames = [
    "IMG_1813",
    "IMG_3227",
    "IMG_3515",
    "IMG_3799",
    "IMG_4700",
    "IMG_6726",
    "IMG_8176",
]

def read_timeline_data(filename):
    jsonfile = os.path.join(base_dir, "PSNR/level-6/time_limit-1200/qscale-0.3/speedup-1.25", f"{filename}_statistics.json")
    with open(jsonfile, 'r') as f:
        data = json.load(f)
        results = []
        for step_results in data["step_results"]:
            results.append(
                Data(
                    psnr=-step_results["best_loss"],
                    legal=step_results["legal"],
                    time=step_results["time"],
                    stage=step_results["stage"],
                    ans=step_results["best_results"],
                )
            )
        return results

In [37]:
from src.fileio import FileIO

qscale_idx = 0

fileio = FileIO(h=4480, w=6720, ctu_size=512, mosaic=False)

In [56]:
def hex_to_bgr(hex_color):
    # Convert hex to RGB tuple
    hex_color = hex_color.lstrip("#")
    rgb = tuple(int(hex_color[i : i + 2], 16) for i in (0, 2, 4))
    # Convert RGB to BGR
    bgr = (rgb[2], rgb[1], rgb[0])
    return bgr

colors_hex = ["#A83411", "#ADA091", "#6D1E0D", "#39150D", "#D6D9D9", "#C4C78D"]
colors = [hex_to_bgr(color) for color in colors_hex]

In [66]:
from typing import Optional
import cv2

def generate_chosen_result_in_img(method_ids, baseimg:Optional[np.ndarray] = None):
    image = np.zeros((4480, 6720, 3), np.uint8)
    for i, bb in zip(method_ids, fileio.block_indexes):
        cv2.rectangle(
            image, (bb[1], bb[0]), (bb[3], bb[2]), color=colors[i], thickness=cv2.FILLED
        )
    if baseimg is not None:
        image = cv2.addWeighted(baseimg, 0.4, image, 0.6, 0)
    for i, bb in zip(method_ids, fileio.block_indexes):
        cv2.rectangle(
            image, (bb[1], bb[0]), (bb[3], bb[2]), color=(0, 0, 0), thickness=16
        )
    return image

In [67]:
# Across images

baseimg = cv2.imread('/home/xyhang/projects/VCIP2023-grand-challenge/images/6720x4480/IMG_6726.png')

libpng warning: iCCP: known incorrect sRGB profile


In [71]:
with open('/home/xyhang/projects/VCIP2023-grand-challenge/experiments/SAv1/visualize/PSNR/level-6/time_limit-1200/qscale-0.3/speedup-1.25/IMG_6726_statistics.json', 'r') as f:
    results = json.load(f)

In [69]:
print(results['step_results'][0])

{'best_legal': True, 'best_loss': -38.568346732390204, 'best_results': [1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'legal': True, 'loss': -38.568346732390204, 'results': [1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'stage': 'init', 'step': 0, 'time': 64.88242385885678}


In [72]:
for i in range(0, len(results['step_results']), 350):
    step_results = results['step_results'][i]
    out_img = generate_chosen_result_in_img(step_results["best_results"], baseimg)
    out_img = cv2.resize(out_img, (6720 // 4, 4480 // 4))
    cv2.imwrite(f"output/step{i}.png", out_img)
    print(f"i; {step_results['best_loss']:.2f}")

i; -38.57
i; -39.33
i; -39.41
i; -39.41
i; -39.41
i; -39.44
i; -39.48
